In [1]:
import os, sys, zarr
sys.path.append('..')
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
from os.path import abspath, dirname
from easi_fish import n5_metadata_utils as n5mu
import warnings
warnings.filterwarnings('ignore')

In [2]:
## input
input_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs"
out_dir = os.path.join(input_dir, "testfull-aug5-v2")  

# fixed image (directory - n5 format)
fix_dir = os.path.join(input_dir, "r2\\export_sigma3.n5")
# get appropriate image data within fix_dir
subpath='\\c3\\s2'

# registered image (directory - n5 format)
reg_dir = os.path.join(input_dir, "r1\\registration\\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\\warped")  

# segmentation mask (tif format accepted here)
lb_dir  = os.path.join(input_dir, "r2\\segmentation\\lt171_gene_4tile_r2-c3.tif" ) 

# output directory
## end input

## output
out_mask = os.path.join(out_dir, 'mask.tif')
out_badroi = os.path.join(out_dir, 'bad_roi_list.npy') 

In [3]:
%%time
#voxel size in µm (x, y, z) (post-expansion)
vox= n5mu.read_voxel_spacing(fix_dir, subpath)
#image size in pixel (x, y, z)
grid=n5mu.read_voxel_grid(fix_dir, subpath)
#image size in physical space (x, y, z) (post-expansion)
size=grid*vox
print('voxel size is:',vox)
print('image size in pixel unit is:',grid)
print('image size in um unit is:',size)

# get appropriate image data
print("loading images...")
# fix = zarr.open(store=zarr.N5Store(fix_dir), mode='r')     
# img1 = fix[subpath][:, :, :]

reg = zarr.open(store=zarr.N5Store(reg_dir), mode='r')     
img2 = reg[subpath][:, :, :]
print("all images loaded")

if not os.path.isdir(out_dir):
    print(out_dir)
    os.mkdir(out_dir)

mask=np.full((grid[2], grid[1], grid[0]),1)
mask[img2==0]=0
imsave(out_mask, mask)
print("mask generated")
print("mask dimension is:", mask.shape)

lb=imread(lb_dir)
roi = np.max(lb)
print(roi)

# # Get list of ROIs that are fully or partially outside the mask 
bad_roi=np.unique(lb[mask==0])
if bad_roi[0] == 0:
    bad_roi = bad_roi[1:]
np.save(out_badroi, bad_roi)
print("# of ROIs rejected:", len(bad_roi))

voxel size is: [0.92 0.92 0.84]
image size in pixel unit is: [ 916  915 1019]
image size in um unit is: [842.72003 841.8     855.95996]
loading images...
all images loaded
D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\testfull-aug5-v2
mask generated
mask dimension is: (1019, 915, 916)
13758
# of ROIs rejected: 891
CPU times: total: 34.2 s
Wall time: 24 s


In [4]:
bad_roi


array([   32,    52,    57,    61,   115,   183,   302,   314,   320,
         324,   370,   396,   455,   505,   525,   577,   618,   725,
         732,   737,   790,   799,   809,   814,   822,   850,   880,
         881,   889,   895,   905,   920,  1004,  1027,  1036,  1051,
        1056,  1089,  1098,  1105,  1132,  1142,  1174,  1185,  1218,
        1231,  1296,  1324,  1454,  1470,  1471,  1473,  1498,  1518,
        1534,  1573,  1575,  1581,  1614,  1652,  1654,  1661,  1774,
        1809,  1818,  1863,  1873,  1896,  1937,  1951,  1957,  2008,
        2013,  2021,  2023,  2044,  2113,  2198,  2241,  2264,  2276,
        2279,  2284,  2351,  2365,  2376,  2394,  2400,  2415,  2482,
        2490,  2519,  2534,  2556,  2562,  2579,  2587,  2615,  2637,
        2665,  2690,  2730,  2809,  2819,  2827,  2860,  2967,  2982,
        2998,  3031,  3091,  3232,  3246,  3260,  3276,  3310,  3334,
        3398,  3405,  3451,  3482,  3488,  3490,  3549,  3551,  3610,
        3634,  3654,